In [ ]:
import cv2
import winsound
import time
import requests


EMERGENCY_CONTACT = "+919876543212"  # Replace with actual mobile number


def get_location():
    try:
        response = requests.get("https://ipapi.co/json/")
        data = response.json()
        latitude = data['latitude']
        longitude = data['longitude']
        city = data['city']
        region = data['region']
        country = data['country_name']
        postal = data['postal']
        return f"{city}, {region}, {country}, {postal} (Lat: {latitude}, Lon: {longitude})"
    except:
        return "Location not available"


face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')


cap = cv2.VideoCapture(0)

sleep_counter = 0
SLEEP_THRESHOLD = 30 
EMERGENCY_THRESHOLD_SEC = 30  

# Simulated engine status
engine_on = False
sleep_start_time = None
emergency_triggered = False

while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)

    eyes_open = False

    for (x, y, w, h) in faces:
        roi_gray = gray[y:y+h, x:x+w]
        eyes = eye_cascade.detectMultiScale(roi_gray)
        if len(eyes) >= 1:
            eyes_open = True
            break

    if eyes_open:
        sleep_counter = 0
        drowsy_status = "Awake"
        sleep_start_time = None
        emergency_triggered = False
    else:
        sleep_counter += 1
        drowsy_status = f"Sleeping ({sleep_counter})"

        if sleep_start_time is None:
            sleep_start_time = time.time()

        if engine_on and sleep_counter > SLEEP_THRESHOLD:
            winsound.Beep(2000, 1000)

        if engine_on and not emergency_triggered:
            sleep_duration = time.time() - sleep_start_time
            if sleep_duration > EMERGENCY_THRESHOLD_SEC:
                print("🚨 EMERGENCY: Driver unresponsive. Calling ambulance and police...")
                location = get_location()
                print(f"🚩 Sending Location: {location}")
                print(f"📞 Contacting emergency number: {EMERGENCY_CONTACT}")
                emergency_triggered = True
                winsound.Beep(2500, 2000)

    engine_status = "Engine: ON" if engine_on else "Engine: OFF"

    cv2.putText(frame, drowsy_status, (30, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.putText(frame, engine_status, (30, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2)
    cv2.imshow("Driver Emergency Safety System", frame)

    key = cv2.waitKey(1) & 0xFF

    if key == ord('q'):
        break
    elif key == ord('e'):
        engine_on = True

cap.release()
cv2.destroyAllWindows()


🚨 EMERGENCY: Driver unresponsive. Calling ambulance and police...
🚩 Sending Location: Chennai, Tamil Nadu, India, 600078 (Lat: 13.0895, Lon: 80.2739)
📞 Contacting emergency number: +919876543212
